# Implementação manual de Perceptron

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

## Dataset: Titanic

Dataset disponível em: https://www.kaggle.com/competitions/titanic/data

In [2]:
df = pd.read_csv('train.csv')
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [3]:
# Visão geral dos dados.
df.info()
df.describe().round(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.00,891.00,891.00,714.00,891.00,891.00,891.00
mean,446.00,0.38,2.31,29.70,0.52,0.38,32.20
std,257.35,0.49,0.84,14.53,1.10,0.81,49.69
min,1.00,0.00,1.00,0.42,0.00,0.00,0.00
25%,223.50,0.00,2.00,20.12,0.00,0.00,7.91
50%,446.00,0.00,3.00,28.00,0.00,0.00,14.45
75%,668.50,1.00,3.00,38.00,1.00,0.00,31.00
max,891.00,1.00,3.00,80.00,8.00,6.00,512.33


In [4]:
# Transformando variáveis categóricas em dummy.
df['female'] = np.where(df['Sex']=='female', 1, 0)
df['embarked_c'] = np.where(df['Embarked'] == 'C', 1, 0)
df['embarked_q'] = np.where(df['Embarked'] == 'Q', 1, 0)
df['embarked_s'] = np.where(df['Embarked'] == 'S', 1, 0)

In [5]:
# Quais variáveis usaremos?
df.corr().round(2) * 100

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,female,embarked_c,embarked_q,embarked_s
PassengerId,100.0,-1.0,-4.0,4.0,-6.0,-0.0,1.0,-4.0,-0.0,-3.0,2.0
Survived,-1.0,100.0,-34.0,-8.0,-4.0,8.0,26.0,54.0,17.0,0.0,-16.0
Pclass,-4.0,-34.0,100.0,-37.0,8.0,2.0,-55.0,-13.0,-24.0,22.0,8.0
Age,4.0,-8.0,-37.0,100.0,-31.0,-19.0,10.0,-9.0,4.0,-2.0,-3.0
SibSp,-6.0,-4.0,8.0,-31.0,100.0,41.0,16.0,11.0,-6.0,-3.0,7.0
Parch,-0.0,8.0,2.0,-19.0,41.0,100.0,22.0,25.0,-1.0,-8.0,6.0
Fare,1.0,26.0,-55.0,10.0,16.0,22.0,100.0,18.0,27.0,-12.0,-17.0
female,-4.0,54.0,-13.0,-9.0,11.0,25.0,18.0,100.0,8.0,7.0,-13.0
embarked_c,-0.0,17.0,-24.0,4.0,-6.0,-1.0,27.0,8.0,100.0,-15.0,-78.0
embarked_q,-3.0,0.0,22.0,-2.0,-3.0,-8.0,-12.0,7.0,-15.0,100.0,-50.0


Variáveis mais relevantes

Avaliando a correlação das variáveis, observamos que as variáveis Pclass, Fare e female apresentam alta correlação. Utilizaremos elas para a construção do perceptron.

In [6]:
lista_var = ['Pclass', 'Fare', 'female']

df[lista_var + ['Survived']].head()

,Pclass,Fare,female,Survived
0,3,7.2500,0,0
1,1,71.2833,1,1
2,3,7.9250,1,1
3,1,53.1000,1,1
4,3,8.0500,0,0


## 1 camada

![perceptron 1 camada](img/perceptron-1camada.PNG)

In [10]:
def fx_sigmoide(x):
    return 1 / (1 + np.exp(-x))

def fx_binary(x):
    if x >= 0:
        return 1
    else:
        return 0

In [ ]:
%matplotlib tk

tx_aprendizado = 0.01
pesos = np.array([1, -.01, 1])
#pesos = np.array([1, 1, 1])
epoch = 100

figure, ax = plt.subplots(figsize=(8, 5))
x = [0] * epoch

for i in range(epoch):
    erro_total = 0

    for index, row in df.sample(frac=1).iterrows():

        # Calculando o somaproduto dos inputs vs pesos.
        somaproduto = (df.loc[index, lista_var] * pesos).sum()

        # Aplicando a função sigmóide.
        res = fx_binary(somaproduto)

        # Calculando a soma dos erros de toda a amostra.
        erro = row['Survived'] - res
        erro_total += abs(erro)

        # Calculando os novos pesos.
        for order, value in enumerate(pesos):
            
            # ********
            pesos[order] += tx_aprendizado * row[lista_var[order]] * erro   # <-- linha mais importante do exercício
            # ********

    # Plot interativo no MatplotLib
    x[i] = erro_total
    if (i >= 1):
        plt.title(f"Perceptron 1 camada: erro durante treinamento\n(menor erro {min(x[0:i])}, interação {x.index(min(x[0:i]))})", fontsize=14)
    else:
        plt.title(f"Perceptron 1 camada: erro durante treinamento\n(menor erro=0)", fontsize=14)

    if i == 0:
        cor = 'gray'
    elif x[i] < x[i-1]:
        cor = 'green'
    else:
        cor = 'red'

    plt.xlabel(f"Interações (atual: {i})")
    plt.ylabel("Erro total")
    plt.xlim(0, epoch)
    plt.ylim(0, 600)
    plt.bar(list(range(epoch)), x, color=cor)
    plt.pause(0.0001)

plt.show()

melhor_erro = min(x[0:i])
melhor_epoch = x.index(min(x[0:i]))

## 2 camadas

![perceptron 2 camada](img/perceptron-2camada.PNG)

In [7]:
df[lista_var + ['Survived']].head()

,Pclass,Fare,female,Survived
0,3,7.2500,0,0
1,1,71.2833,1,1
2,3,7.9250,1,1
3,1,53.1000,1,1
4,3,8.0500,0,0


In [11]:
fx_sigmoide(10)

def fx_der_sigmoide(x):
    return fx_sigmoide(x) * (1 - fx_sigmoide(x))

In [14]:
%matplotlib tk

epoch = 100
momentum = 1
tx_aprend = 0.05
pesos1 = np.array([[0.1, 0.1, 0.1],
                  [0.2, 0.2, 0.2]])
pesos2 = np.array([0.1, 0.1])
figure, ax = plt.subplots(figsize=(8, 5))
y = [0] * epoch

for i in range(epoch):
    
    # Camada 1 -> Camada 2: cálculo do neurônio de "cima".
    sp_cima = (df[lista_var] * pesos1[0]).sum(axis=1)
    output_cima = sp_cima.apply(fx_sigmoide) 

    # Camada 1 -> Camada 2: cálculo do neurônio de "baixo".
    sp_baixo = (df[lista_var] * pesos1[1]).sum(axis=1)
    output_baixo = sp_baixo.apply(fx_sigmoide) 
    # obs: as etapas anteriores também podem ser feitas matricialmente utilizando-se np.dot(input * pesos).

    # Camada 2 -> Camada final
    output2 = pd.DataFrame({'cima': output_cima, 'baixo': output_baixo})
    sp = (output2 * pesos2).sum(axis=1)
    output = sp.apply(fx_sigmoide)

    # Calculando o erro.
    erro = df['Survived'] - output
    erro_epoch = abs(erro).mean()

    # Cálculo do delta das camadas de saídas e intermediária.
    # obs: delta é um parâmetro que orientará o gradiente descendente a escolher o caminho com menor função erro.
    delta_saida = erro * erro.apply(fx_der_sigmoide)
    delta_cima = output_cima.apply(fx_der_sigmoide) * pesos2[0] * delta_saida
    delta_baixo = output_baixo.apply(fx_der_sigmoide) * pesos2[1] * delta_saida

    # Cálculo dos pesos com backpropagation. 
    # obs: a fórmula é peso_novo = peso_velho * momentum + (entrada * delta * tx_aprendizagem)
    # ********
    pesos2 = pesos2 * momentum + (np.dot(output2.T, delta_saida) * tx_aprend)   #<-- linha mais importante
    pesos1 = pesos1 * momentum + (np.dot(df[lista_var].T, np.array([delta_cima, delta_baixo]).T).T * tx_aprend)
    # ********

     # Plot interativo no MatplotLib
    y[i] = erro_epoch
    if (i >= 1):
        plt.title(f"Perceptron 2 camadas: erro durante treinamento {erro_epoch}\n(menor erro {min(y[0:i])}, interação {y.index(min(y[0:i]))})", fontsize=14)
    else:
        plt.title(f"Perceptron 2 camadas: erro durante treinamento\n(menor erro=0)", fontsize=14)

    if i == 0:
        cor = 'gray'
    elif y[i] < y[i-1]:
        cor = 'green'
    else:
        cor = 'red'

    plt.xlabel(f"Interações (atual: {i})")
    plt.ylabel("Erro total")
    plt.xlim(0, epoch)
    #plt.ylim(0, 1)
    plt.bar(list(range(epoch)), y, color=cor)
    plt.pause(0.0001)

plt.show()